In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import numpy as np
import pandas as pd

TensorFlow version: 2.8.0


In [ ]:
# linear case y= x*a + b
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
xs = np.array([-1, 0, 1, 2, 3, 4], dtype=float)
ys = np.array([-3, -1, 1, 3, 5, 7], dtype=float)

model.fit(xs, ys, epochs=1000)
print(model.predict([10.0]))

In [ ]:
#visualize mnist the data
import matplotlib.pyplot as plt

idx = 0
np.set_printoptions(linewidth=320)

fs_mnist = tf.keras.datasets.fashion_mnist
(train_imgs, train_labels), (test_imgs, test_labels) = fs_mnist.load_data()
print(f'label: {train_labels[idx]}')
print(f'\n image: \n {train_imgs[idx]}')

plt.imshow(train_imgs[idx])

In [ ]:
class myModel:
  def denseModel(self):
      self.model = tf.keras.Sequential([
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation=tf.nn.relu),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

  def modelConv(self):
      self.model = tf.keras.Sequential([
      #conv layers
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      #dense layers
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation=tf.nn.relu),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)           
    ])

In [ ]:
 # callback
class callBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss') < 0.4):
      print("\n the loss is less then 0.4, so the training is canceled")
      self.model.stop_training = True


In [ ]:
callBack = callBack()
fs_mnist = tf.keras.datasets.fashion_mnist
(train_imgs, train_labels), (test_imgs, test_labels) = fs_mnist.load_data()

# normalize data
train_imgs = train_imgs / 255.0
test_imgs = test_imgs / 255.0

#build model
model = tf.keras.Sequential([
  #conv layers
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  #dense layers
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)           
])

from tensorflow.python.eager.monitoring import Metric
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#fit and evaluate
model.fit(train_imgs, train_labels, epochs=10, callbacks=[callBack])
model.evaluate(test_imgs, test_labels)

In [ ]:
classification = model.predict(test_imgs)
print(classification[0])

In [ ]:
#visualize the conv filters
import matplotlib.pyplot as plt
from tensorflow.keras import models

print(f'test label:{test_labels[:100]}')

f, axarr = plt.subplots(3,4)
FIRST_IMG = 0
SECOND_IMG = 23 
THIRD_IMG = 28 
CONV_NUM = 1 

layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)

for x in range(0,4):
  f1 = activation_model.predict(test_imgs[FIRST_IMG].reshape(1,28,28,1))[x]
  print(f1.shape)
  axarr[0,x].imshow(f1[0, :, :, CONV_NUM], cmap='inferno')
  axarr[0,x].grid(False)

  f2 = activation_model.predict(test_imgs[SECOND_IMG].reshape(1,28,28,1))[x]
  axarr[1,x].imshow(f2[0, :, :, CONV_NUM], cmap='inferno')
  axarr[1,x].grid(False)

  f3 = activation_model.predict(test_imgs[THIRD_IMG].reshape(1,28,28,1))[x]
  axarr[2,x].imshow(f3[0, :, :, CONV_NUM], cmap='inferno')
  axarr[2,x].grid(False)


In [ ]:
# visualize affects of conv filters
from scipy import misc
import matplotlib.pyplot as plt
import numpy as np


# load the ascent image
ascent_image = misc.ascent()

# Copy image to a numpy array
image_transformed = np.copy(ascent_image)

# Get the dimensions of the image
size_x = image_transformed.shape[0]
size_y = image_transformed.shape[1]

# Experiment with different values and see the effect
filter = [ [0, 1, 0], [1, -4, 1], [0, 1, 0]]

# A couple more filters to try for fun!
filter = [ [-1, -2, -1], [0, 0, 0], [1, 2, 1]]
filter = [ [-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]

# If all the digits in the filter don't add up to 0 or 1, you 
# should probably do a weight to get it to do so
# so, for example, if your weights are 1,1,1 1,2,1 1,1,1
# They add up to 10, so you would set a weight of .1 if you want to normalize them
weight  = 1

# Iterate over the image
for x in range(1,size_x-1):
  for y in range(1,size_y-1):
      convolution = np.sum(ascent_image[(x-1):(x+2):1, (y-1):(y+2):1] * filter)
      
      # Multiply by weight
      convolution = convolution * weight   
      
      # Check the boundaries of the pixel values
      if(convolution<0):
        convolution=0
      if(convolution>255):
        convolution=255

      # Load into the transformed image
      image_transformed[x, y] = convolution

# Assign dimensions half the size of the original image
new_x = int(size_x/2)
new_y = int(size_y/2)

# Create blank image with reduced dimensions
newImage = np.zeros((new_x, new_y))

# Iterate over the image
for x in range(0, size_x, 2):
  for y in range(0, size_y, 2):
    
    # Store all the pixel values in the (2,2) pool
    pixels = []
    pixels.append(image_transformed[x, y])
    pixels.append(image_transformed[x+1, y])
    pixels.append(image_transformed[x, y+1])
    pixels.append(image_transformed[x+1, y+1])

    # Get only the largest value and assign to the reduced image
    newImage[int(x/2),int(y/2)] = max(pixels)

# Visualize the image
plt.grid(False)
plt.gray()
plt.axis('off')
plt.imshow(ascent_image)
plt.show()

plt.gray()
plt.grid(False)
plt.imshow(image_transformed)
plt.show()  

plt.gray()
plt.grid(False)
plt.imshow(newImage)
plt.show()     